In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Loading the Dataset

In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
gender = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
df_org = df.copy

## Filling Missing Values

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df['titles'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,titles
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [7]:
title_pclass_age = df.groupby(['titles','Pclass','Sex'])['Age'].median()

for (titles,pclass,sex),age in title_pclass_age.items():
    df.loc[(df['titles'] == titles) & (df['Pclass'] == pclass) & (df['Age'].isnull()) & (df['Sex'] == sex),'Age'] = age

In [8]:
df.loc[df['Embarked'].isnull(),'Embarked'] = 'S'
df["Embarked"].value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

In [9]:
df = df.drop("Cabin", axis=1)

In [10]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,titles
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr


In [11]:
test = test.drop("Cabin", axis=1)

In [12]:
test['titles'] = test['Name'].str.extract(' ([A-Za-z]+)\.',expand = False)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,titles
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Mrs


In [13]:
title_pclass_age = test.groupby(['titles','Pclass','Sex'])['Age'].median()

for (titles,pclass,sex),age in title_pclass_age.items():
    test.loc[(test['titles'] == titles) & (test['Pclass'] == pclass) & (test['Age'].isnull()) & (test['Sex'] == sex),'Age'] = age

In [14]:
test[test['Age'].isnull()]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,titles
88,980,3,"O'Donoghue, Ms. Bridget",female,NaN,0,0,364856,7.75,Q,Ms


In [15]:
test['Age'].mean()

29.408872901678656

In [16]:
test.loc[test['Age'].isnull(),'Age'] = 29

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Embarked     418 non-null    object 
 10  titles       418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [18]:
test['Fare'].mean()

35.627188489208635

In [19]:
test.loc[test['Fare'].isnull(),'Fare'] = 35

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Embarked     418 non-null    object 
 10  titles       418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
 11  titles       891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [22]:
df = df.drop(['PassengerId','Name','Ticket'],axis = 1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,titles
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


In [23]:
df['Sex'] = df['Sex'].map({'male':0,'female':1})
test['Sex'] = test['Sex'].map({'male':0,'female':1})

In [24]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop = 'first' , sparse = False)
ohe.fit(df[['Embarked']])

train_embarked = ohe.transform(df[['Embarked']])
test_embarked = ohe.transform(test[['Embarked']])

train_embarked_df = pd.DataFrame(train_embarked, columns=ohe.get_feature_names_out(['Embarked']), index=df.index)
test_embarked_df = pd.DataFrame(test_embarked, columns=ohe.get_feature_names_out(['Embarked']), index=test.index)

df = pd.concat([df.drop('Embarked', axis=1), train_embarked_df], axis=1)
test = pd.concat([test.drop('Embarked', axis=1), test_embarked_df], axis=1)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
ohe = OneHotEncoder(drop = 'first',sparse = False,handle_unknown='ignore')
ohe.fit(df[['titles']])
train_titles = ohe.transform(df[['titles']])
test_titles = ohe.transform(test[['titles']])

train_titles_df = pd.DataFrame(train_titles , columns = ohe.get_feature_names_out(['titles']), index = df.index)
test_titles_df = pd.DataFrame(test_titles , columns = ohe.get_feature_names_out(['titles']), index = test.index)

df = pd.concat([df.drop('titles',axis = 1),train_titles_df] , axis = 1)
test = pd.concat([test.drop('titles' , axis = 1) , test_titles_df],axis = 1)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## Numerical Features Scaling

In [26]:
from sklearn.preprocessing import StandardScaler
row = ['Age', 'Fare']
scaler = StandardScaler()
a = scaler.fit_transform(df[row])
b = scaler.transform(test[row])

## Model Training

In [27]:
from sklearn.model_selection import train_test_split
y = df['Survived']
X = df.drop('Survived', axis=1)
features = X.columns

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

## 1. LOGISTIC REGRESSION

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(max_iter = 1000 ,random_state = 42)
log_reg.fit(X_train,y_train)

y_pred = log_reg.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"LOGISTIC REGRESSION: {acc:.2f}")

LOGISTIC REGRESSION: 0.82


## 2. RANDOM FOREST CLASSIFIER

In [29]:
from sklearn.ensemble import RandomForestClassifier

rfr = RandomForestClassifier(n_estimators = 100 , random_state = 42)

rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)

acc = accuracy_score(y_test,y_pred)
print(f"RANDOM FOREST CLASSIFIER: {acc:.2f}")

RANDOM FOREST CLASSIFIER: 0.84


## 3. DECISION TREE CLASSIFIER

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_entropy = DecisionTreeClassifier(
    criterion='log_loss',
    max_depth= None,
    min_samples_split = 2,
    min_samples_leaf = 1,
    random_state=42
)

tree_entropy.fit(X_train, y_train)

y_pred = tree_entropy.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"DECISION TREE CLASSIFIER: {acc:.2f}")

DECISION TREE CLASSIFIER: 0.80


## HYPERPARAMETER TUNING

**1. Logistic Regression****

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("log_reg", LogisticRegression(solver="lbfgs", max_iter=1000))
])

param_grid = {
    "log_reg__C": [0.01, 0.1, 1, 10, 100],  
    "log_reg__penalty": ["l1","l2"],
    "log_reg__solver": ["liblinear", "saga"]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print(f"Best CV Score:, {grid.best_score_:.2f}")

y_pred = grid.predict(X_test)
A = accuracy_score(y_test, y_pred)
print(f"Random Forest Classifier After Hyperparmeter Tuning : {A:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Best Params: {'log_reg__C': 0.1, 'log_reg__penalty': 'l2', 'log_reg__solver': 'saga'}
Best CV Score:, 0.83
Random Forest Classifier After Hyperparmeter Tuning : 0.81


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**2. Random Forest Classifier**

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

rs = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,   
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

model = rs.fit(X_train, y_train)
print("Best Parameters:", model.best_params_)
print(f"Best CV Score : {model.best_score_:.2f} ")

best_model = rs.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Random Forest Classifier After Hyperparmeter Tuning : {acc:.2f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': False}
Best CV Score : 0.84 
Random Forest Classifier After Hyperparmeter Tuning : 0.84


In [33]:
best_model = model.best_estimator_
best_model

RandomForestClassifier(bootstrap=False, max_depth=20, max_features='log2',
                       min_samples_leaf=2, min_samples_split=10,
                       random_state=42)

**3. Decision Tree Classifier**

In [34]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

AC = accuracy_score(y_test, y_pred)
print(f"ADABOOST CLASSIFIER: {AC:.3f}")

ADABOOST CLASSIFIER: 0.810


In [35]:
best_model.fit(X, y)
X_final_test = test[features] 

final_preds = best_model.predict(X_final_test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": final_preds
})

submission["Survived"] = submission["Survived"].astype(int)
submission.to_csv("submission.csv", index=False) 

In [36]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [37]:
import os
print(os.getcwd())
print(os.listdir())

/kaggle/working
['submission.csv', '__notebook__.ipynb']


In [38]:
print(final_preds[:10])
print(len(final_preds), len(test))

[0 0 0 0 1 0 0 0 1 0]
418 418


In [39]:
!ls -l /kaggle/working

total 88
---------- 1 root root 82753 Sep  3 06:39 __notebook__.ipynb
-rw-r--r-- 1 root root  2839 Sep  3 06:39 submission.csv
